In [2]:
# Imports

import os
import string
import re
from collections import Counter
import html  

from pprint import pprint

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from cltk.corpus.latin import latinlibrary
from cltk.tokenize.line import LineTokenizer
from cltk.tokenize.word import WordTokenizer
from cltk.stem.latin.j_v import JVReplacer

In [3]:
# Setup CLTK tools

line_tokenizer = LineTokenizer('latin')
word_tokenizer = WordTokenizer('latin')
replacer = JVReplacer()

## Get text of the Pliny 10

In [4]:
# Get the Pliny Ep. 10 text

files = latinlibrary.fileids()
pliny_ep10_raw = latinlibrary.raw('pliny.ep10.txt')
print(pliny_ep10_raw)

Pliny the Younger
		 

		 
		 
	 

 

 C. PLINII CAECILII SECVNDI EPISTVLARVM LIBER DECIMVS AD TRAIANVM IMPERATOREM CVM EIVSDEM RESPONSIS
 

 

 
 1 
 

 
C. PLINIUS TRAIANO IMPERATORI
 

 
 1 Tua quidem pietas, imperator sanctissime, optaverat, ut quam tardissime succederes patri; sed di immortales festinaverunt virtutes tuas ad gubernacula rei publicae quam susceperas admovere. 2 Precor ergo ut tibi et per te generi humano prospera omnia, id est digna saeculo tuo contingant. Fortem te et hilarem, imperator optime, et privatim et publice opto.
 

 
 2 
 

 
C. PLINIUS TRAIANO IMPERATORI
 

 
 1 Exprimere, domine, verbis non possum, quantum mihi gaudium attuleris, quod me dignum putasti iure trium liberorum. Quamvis enim Iuli Serviani, optimi viri tuique amantissimi, precibus indulseris, tamen etiam ex rescripto intellego libentius hoc ei te praestitisse, quia pro me rogabat. 2 Videor ergo summam voti mei consecutus, cum inter initia felicissimi principatus tui probaveris me ad peculia

In [5]:
# Preprocess texts

def preprocess(text):

    remove_list = [r'\bPliny the Younger\b',
                   r'\bThe Latin Library\b',
                   r'\bThe Classics Page',
                   r'C. PLINII CAECILII SECVNDI EPISTVLARVM LIBER DECIMVS AD TRAIANVM IMPERATOREM CVM EIVSDEM RESPONSIS',
                  ]
    
    for pattern in remove_list:
        text = re.sub(pattern, '', text)
    
    # REMOVE LETTER HEADINGS?
    
    text = html.unescape(text) # Handle html entities
    text = re.sub(r'&nbsp;?', ' ',text) #&nbsp; stripped incorrectly in corpus?
    text = re.sub(r'\x00',' ',text) #Another space problem?
    
    # Remove roman numeral headings; must be before lower & replacer
    #text = re.sub(r'\b(M{1,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})|M{0,4}(CM|C?D|D?C{1,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,4})|M{0,4}(CM|CD|D?C{0,3})(XC|X?L|L?X{1,3})(IX|IV|V?I{0,3})|M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|I?V|V?I{1,3}))\b[\.]',' ',text)
    
    text = text.lower()
    text = replacer.replace(text) #Normalize u/v & i/j
    
    # Herdan also normalizes 'qu' to 'c'
    #text = re.sub('qu', 'k', text)
    #text = re.sub('cui', 'ku', text)
    
    
    punctuation ="\"#$%&\'()*+,-/:;<=>@[\]^_`{|}~.?!«»"
    translator = str.maketrans({key: " " for key in punctuation})
    text = text.translate(translator)
    
    translator = str.maketrans({key: " " for key in '0123456789'})
    text = text.translate(translator)
    
    # Remove letters standing alone on a line as leftover indicators
    text = re.sub(r'^\s*?[a-z]\s*?$','', text, flags=re.MULTILINE)

    # Trim spaces around lines
    text = re.sub(r'^\s*','', text, flags=re.MULTILINE)
    text = re.sub(r'\s*$','', text, flags=re.MULTILINE)

    text = re.sub(r'\n',r' ', text, flags=re.MULTILINE) # Replace multiple new lines with one
    text = re.sub('[ ]+',' ', text) # Remove double spaces

    # Remove text of Trajan's responses to Pliny, leaving nothing
    # Couldn't get this to consistently insert \n before \g<1>
    text = re.sub(r'traianus plinio.*?(c plinius traiano imperatori)',r'\g<1>',text, flags=re.DOTALL)

    # Split text on Pliny's salutation to Trajan, leaving each letter in one paragraph
    text = re.sub(r'c plinius traiano imperatori ',r'\n\n', text)

    return text

print(preprocess(pliny_ep10_raw))



tua quidem pietas imperator sanctissime optauerat ut quam tardissime succederes patri sed di immortales festinauerunt uirtutes tuas ad gubernacula rei publicae quam susceperas admouere precor ergo ut tibi et per te generi humano prospera omnia id est digna saeculo tuo contingant fortem te et hilarem imperator optime et priuatim et publice opto 

exprimere domine uerbis non possum quantum mihi gaudium attuleris quod me dignum putasti iure trium liberorum quamuis enim iuli seruiani optimi uiri tuique amantissimi precibus indulseris tamen etiam ex rescripto intellego libentius hoc ei te praestitisse quia pro me rogabat uideor ergo summam uoti mei consecutus cum inter initia felicissimi principatus tui probaueris me ad peculiarem indulgentiam tuam pertinere eoque magis liberos concupisco quos habere etiam illo tristissimo saeculo uolui sicut potes duobus matrimoniis meis credere sed di melius qui omnia integra bonitati tuae reseruarunt malui hoc potius tempore me patrem fieri quo futurus

In [ ]:
# WRITE SCRIPT TO SEPARATE LETTERS

In [19]:
# PLACEHOLDER FOR Preprocess the Pliny Ep. 10 texts
# print(preprocess(pliny_ep10_raw))


c plinii caecilii secundi epistularum liber decimus ad traianum imperatorem cum eiusdem responsis
c plinius traiano imperatori
tua quidem pietas imperator sanctissime optauerat ut quam tardissime succederes patri sed di immortales festinauerunt uirtutes tuas ad gubernacula rei publicae quam susceperas admouere precor ergo ut tibi et per te generi humano prospera omnia id est digna saeculo tuo contingant fortem te et hilarem imperator optime et priuatim et publice opto
c plinius traiano imperatori
exprimere domine uerbis non possum quantum mihi gaudium attuleris quod me dignum putasti iure trium liberorum quamuis enim iuli seruiani optimi uiri tuique amantissimi precibus indulseris tamen etiam ex rescripto intellego libentius hoc ei te praestitisse quia pro me rogabat uideor ergo summam uoti mei consecutus cum inter initia felicissimi principatus tui probaueris me ad peculiarem indulgentiam tuam pertinere eoque magis liberos concupisco quos habere etiam illo tristissimo saeculo uolui s